# BK figures
This code will create the figures for BK model reduction using MBAM for finding boundaries, and manual model reduction

## BK full model reduction

In [ ]:
# The code used to produce the output from reducing the 8 parameter MBAM model to 3 parameters
#= Commented out
include("BK_functions/bk_setup_script.jl"); 
outp = MBAM(BK_simulator, phi0, x_grid, model_iters=[10, 11, 12, 13, 14],
       move_dir=[-1, -1, 1, 1, -1], boundary_time=10, verbose=1, reduce_func=BK_reduce);
=#

# Saved into file  Figures/BK_manual_model_10_to_14_reduct.jld", "outp"

In [2]:
# Load the saved data
using JLD
outp = load("BK_manual_model_10_to_14_reduct.jld", "outp");

# The output is an array containing [phis, costs, Ds, Vs, t_out, y_out, model_reduced]
# each of those contain all information for each iteration, so out[1][3] would be phi values of the 3rd model iteration


In [151]:
# Load the BK functions and the x_grid(Ca, V)
include("../BK_functions/bk_setup_script.jl"); 
Ca = reshape(hcat(x_grid...)[1,:], (8,13));
V = reshape(hcat(x_grid...)[2,:], (8,13));

In [109]:
# Plot the model output at phi0
using Plots
f_model(phi; model_id=0) = reshape(BK_simulator(phi0, hcat(x_grid...),model_id=model_id), (8,13));

Plots.plot(V', f_model(phi0)')

In [63]:
# Now look at the model output using the parameters from the last model reduction iteration (4 parameters)

Plots.plot(V', reshape(BK_simulator(exp(outp[1][5,1]), hcat(x_grid...),model_id=14), (8,13))')

In [165]:
# Plotting parameter value versus time
param_names = Array(Any,length(outp[1]));
param_names[1] = ["L0", "zL", "J0", "zJ", "Kd", "C", "D", "E"]'
Plots.plot(outp[5][1],hcat(outp[6][1]...)[1:8,:]', xlabel="time", xlims=(0,4.2), label=param_names[1]) #t vs all

In [201]:
# Now plot the geodesic courses, from iteration n1, for parameter p1 being fix and plotting the rest
#Iteration one
Plots.plot(hcat(outp[6][1]...)[[1; 3:8],:]', hcat(outp[6][1]...)[2,:]', 
xlabel="log(\phi_i)", ylabel="log(zL)", xlim=(-20,6), label=param_names[1][[1;3:8]']) #all vs zL

In [196]:
#Iteration two
param_names[2] = ["L0", "J0", "zJ", "Kd", "C", "D", "E"]'
Plots.plot(hcat(outp[6][2]...)[[2:7],:]', hcat(outp[6][2]...)[1,:]', 
xlabel="log(\phi_i)", ylabel="log(L0)", label=param_names[2][[2:7]']) #all vs L0

In [169]:
#Iteration three
param_names[3]=["J0", "zJ", "Kd", "L0^1/4 * C"," L0^1/4 * D", "E / L0^1/4"]';
Plots.plot(hcat(outp[6][3]...)[[1:5],:]', hcat(outp[6][3]...)[6,:]', 
xlabel="log(\phi_i)", ylabel="log(E / L0^1/4)", xlims=(-11,10), label=param_names[3][[1:5]']) #all vs L0

In [172]:
#Iteration four
param_names[4]=["J0", "zJ", "Kd", "C * E"," L0^1/4 * D"]';
Plots.plot(hcat(outp[6][4]...)[[2:5],:]', hcat(outp[6][4]...)[1,:]', 
xlabel="log(\phi_i)", ylabel="log(J0)", xlims=(-11,10), label=param_names[4][[2:5]']) #all vs L0